In [1]:
import pandas as pd
import numpy as np

## TO DO
Fix the total savings products caused by adding 1 to random number function   
Fix the total savings balance  
Add probability distribution to closed accounts to make high numbers less likely

### Fetch names and save locally

In [ ]:
# top 100 male and female names in the 2010s from the social security administration
first_names_page = pd.read_html('https://www.ssa.gov/oact/babynames/decades/names2010s.html')

# wikipedia page for popular last names in multiple countries in North America
#last_names_page = pd.read_html('https://en.wikipedia.org/wiki/List_of_most_common_surnames_in_North_America')

# Census with top 1000 American surnames in 2010
last_names_table = pd.read_excel('https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx')

In [ ]:
first_name_table = first_names_page[0]
#last_names_table = last_names_page[8] # uncomment if you're using the wikipedia page

In [ ]:
first_names_cols = ['rank', 'male_name', 'male_number', 'female_name', 'female_number']
first_name_table.columns = first_names_cols
first_name_table = first_name_table[:200]

In [ ]:
males = first_name_table['male_name']
females = first_name_table['female_name']
all_names = males.append(females)

In [ ]:
last_names = last_names_table.iloc[2:1000,0]
lln=list(last_names)
for i,x in enumerate(lln):
    lln[i] = x.lower().capitalize()
clean_last_names = pd.DataFrame(lln)

In [ ]:
clean_last_names.to_csv('last_names.csv', index = False, header=True)
all_names.to_csv('first_names.csv', index = False, header = True)

### Membership_participant

In [5]:
def make_participation_type(portion_101, total_individuals_desired):
    # probability of 101, not portion of 101
    from numpy.random import choice
    participation_types = [101, 102]
    portion_102 = 1-portion_101
    participation_probability = [portion_101, portion_102]
    participation_type = choice(participation_types, total_individuals_desired, p=participation_probability )
    return(list(participation_type))

def make_participation_type_table(participation_type_list):
    participation_type_list = list(participation_type_list)
    n_primaries = len([x for x in participation_type_list if x == 101])
    n_secondaries = len(participation_type_list)-n_primaries
    member_numbers = list(np.random.randint(1000,9000, n_primaries))
    account_type = [101]*len(member_numbers)
    secondary_account_type = [102]*n_secondaries
    account_type += secondary_account_type
    secondary_accounts = list(np.random.choice(member_numbers, n_secondaries, replace = True))
    member_numbers += secondary_accounts
    
    #make random individual ids 
    individual_id = list(np.random.randint(100,10000,len(participation_type_list)))
    
    #make final dataframe
    membership_participation = pd.DataFrame(member_numbers, columns = ['member_nbr'])
    membership_participation['participation_type']=account_type
    membership_participation['individual_id']=individual_id
    
    return (membership_participation)

def make_more_primary_memberships(membership_participation_df, portion_new_memberships):
    n = int(len(membership_participation_df) * portion_new_memberships)
    inds = list(membership_participation_df['individual_id'])
    duplicate_individuals = list(np.random.choice(inds, n, replace=True))
    new_memberships = list(np.random.randint(1000,9000, n))
    participation_type = [101]*n
    more_memberships = pd.DataFrame(new_memberships, columns = ['member_nbr'])
    more_memberships['participation_type']=participation_type
    more_memberships['individual_id']=duplicate_individuals
    
    membership_participation_df = membership_participation_df.append(more_memberships).reset_index(drop=True)
    
    return( membership_participation_df)


In [6]:
ptype = make_participation_type(.5, 10)
mp = make_participation_type_table(ptype)
mp

,member_nbr,participation_type,individual_id
0,8784,101,8471
1,5218,101,8064
2,6368,101,9376
3,3698,101,6623
4,4933,101,4485
5,5218,102,6009
6,6368,102,4121
7,6368,102,9229
8,3698,102,5106
9,3698,102,2698


In [11]:
participation_type = make_more_primary_memberships(mp, .5)
participation_type

,member_nbr,participation_type,individual_id
0,8784,101,8471
1,5218,101,8064
2,6368,101,9376
3,3698,101,6623
4,4933,101,4485
5,5218,102,6009
6,6368,102,4121
7,6368,102,9229
8,3698,102,5106
9,3698,102,2698


### Individual Table

In [ ]:
fn = pd.read_csv('first_names.csv')
ln = pd.read_csv('last_names.csv')
fn = list(fn['0'])
ln = list(ln['0'])

In [8]:
def load_name_lists():
    try:
        fn = pd.read_csv('first_names.csv')
        ln = pd.read_csv('last_names.csv')
        fn = list(fn['0'])
        ln = list(ln['0'])
    # add steps to download names
    except:
        return('error in loading names')
        #fn = pd.read_csv()
        #ln = pd.read_csv()
    return(fn, ln)

def make_dates(length):
    import datetime
    start = datetime.date(2000,1,1)
    end = datetime.date.today()
    dates = [np.random.random() * (end-start)+ start for _ in range(length)] #https://stackoverflow.com/questions/50165501/generate-random-list-of-timestamps-in-python
    for i, x in enumerate(dates):
        dates[i]=str(x)
    return(dates)

def make_name_key(length):
    first_name_key = list(np.random.randint(0,400,length))
    last_name_key = list(np.random.randint(0,1000,length))
    return(first_name_key, last_name_key)

def generate_names(first_name_key, last_name_key):
    fn, ln = load_name_lists()
    
    for i,x in enumerate(first_name_key):
        first_name_key[i]=fn[x]
    
    for i, x in enumerate(last_name_key):
        last_name_key[i]=ln[x]
    
    names = pd.DataFrame(first_name_key, columns = ['first_name'])
    names['last_name']=last_name_key
    
    return(names)

def get_individual_ids(membership_participation_table):
    individual_ids = set(membership_participation_table['individual_id'])
    return(individual_ids)

def create_individual_table(membership_participation_table):
    # read in the membership participation table to get the individual ids and determine how many names to generate
    individual_ids = get_individual_ids(membership_participation_table)
    n = len(individual_ids)
    
    #name keys
    fnk, lnk = make_name_key(n)
    
    #names
    individual = generate_names(fnk, lnk)
    
    #add dates
    open_date = make_dates(n) #add function to ensure open date is suitable time after DOB
    #dob = make_dates(n)
    
    #add attributes to names
    #individual['dob']=dob
    individual['open_date']=open_date
    individual['individual_id']=individual_ids
    
    return(individual)

In [12]:
individual = create_individual_table(more)
individual

,first_name,last_name,open_date,individual_id
0,Kayla,Woodard,2003-10-07,8064
1,Grayson,Arroyo,2008-03-02,9376
2,Greyson,Fitzgerald,2000-07-12,4485
3,Kenneth,Delarosa,2016-01-29,2698
4,Jesus,Mckenzie,2005-07-17,9229
5,Benjamin,Sexton,2000-11-26,5106
6,Xander,Mccarty,2020-12-03,8471
7,Austin,Weeks,2016-01-30,6009
8,Leilani,Acosta,2000-03-07,4121
9,Alyssa,Franklin,2009-08-07,6623


In [ ]:
#return random name
r = np.random.randint(0,10)
r2 =r+1
fnk, lnk = make_name_key(10)
names = generate_names(fnk, lnk)
names[r:r2]

### agr_membertotal_today

In [17]:
import random

values = list(np.linspace(1,90,90))
probabilities = list(np.linspace(1,100,90))

def agr_membertotal_row(member_number):
    row = []
    row.append(member_number)
    
    #first four columns = 1-4
    cls_ln_cnt = np.random.randint(0,5)
    cls_sv_cnt = np.random.randint(0,5)
    opn_ln_all_cnt = np.random.randint(0,5)
    opn_sv_all_cnt = np.random.randint(1,5)
    
    #open loan balance 5
    open_loan_bal = sum(np.random.randint(1000,500000,opn_ln_all_cnt))
    
    #open saving balance 6
    open_sv_bal = sum(np.random.randint(25,100000,opn_sv_all_cnt))
    
    #dividend year to date 7
    div_ytd_amt = open_sv_bal * .01
    
    #interest year to date 8
    int_ytd_amt = open_loan_bal * .05
    
    #savings share deposit balance 9
    SV_SD_BAL = np.random.randint(25,open_sv_bal)

    #savings share deposit count 10
    SV_SD_CNT = np.random.randint(1,opn_sv_all_cnt+1)
    
    #savings certificate balance 11
    SV_CERT_BAL = open_sv_bal - SV_SD_BAL  
    
    #savings certificate count 12
    SV_CERT_CNT = opn_sv_all_cnt - SV_SD_CNT
    
    #max days delinquent 13
    if cls_ln_cnt > 0:
        max_days_delnq_cnt = random.choices(values, cum_weights=probabilities, k=1)[0]
    else:
        max_days_delnq_cnt = 0
    
    #max months delq 14
    max_mths_delnq_cnt = round(int(max_days_delnq_cnt)/30)
    
    #written off loan count 15
    if max_days_delnq_cnt>89:
        wo_ln_cnt = cls_ln_cnt * np.random.randint(1,5)
    else:
        wo_ln_cnt = 0
        
    #written off loan balance 16
    if wo_ln_cnt > 0:
        wo_ln_bal = wo_ln_cnt * np.random.randint(1000,500000)
    else:
        wo_ln_bal = 0
        
    columns = (
    cls_ln_cnt,
    cls_sv_cnt,
    opn_ln_all_cnt,
    opn_sv_all_cnt,
    open_loan_bal,
    open_sv_bal,
    div_ytd_amt,
    int_ytd_amt,
    SV_SD_BAL,
    SV_SD_CNT,
    SV_CERT_BAL,
    SV_CERT_CNT,
    max_days_delnq_cnt,
    max_mths_delnq_cnt,
    wo_ln_cnt,
    wo_ln_bal      
    )
    
    for variable in columns:
        row.append(variable)
    
    return(row)

def generate_agr_membertotal(list_of_member_numbers): 
    col_headers = [
    'member_nbr',
    'cls_ln_cnt',
    'cls_sv_cnt',
    'opn_ln_all_cnt',
    'opn_sv_all_cnt',
    'open_loan_bal',
    'open_sv_bal',
    'div_ytd_amt',
    'int_ytd_amt',
    'sv_sd_bal',
    'sv_sd_cnt',
    'sv_cert_bal',
    'sv_cert_cnt',
    'max_days_delnq_cnt',
    'max_mths_delnq_cnt',
    'wo_ln_cnt',
    'wo_ln_bal'] 
    
    data = []
    
    for member_nbr in list_of_member_numbers:
        data.append(agr_membertotal_row(member_nbr))
    
    data = pd.DataFrame(data, columns = col_headers)
    
    return(data)

In [22]:
member_nbr = list(participation_type['member_nbr'])

agr_membertotal_today = generate_agr_membertotal(member_nbr)


In [19]:
agr_membertotal_today

,member_nbr,cls_ln_cnt,cls_sv_cnt,opn_ln_all_cnt,opn_sv_all_cnt,open_loan_bal,open_sv_bal,div_ytd_amt,int_ytd_amt,sv_sd_bal,sv_sd_cnt,sv_cert_bal,sv_cert_cnt,max_days_delnq_cnt,max_mths_delnq_cnt,wo_ln_cnt,wo_ln_bal
0,8784,0,3,4,2,1563141,135991,1359.91,78157.05,9179,2,126812,0,0.0,0,0,0
1,5218,4,1,2,1,382734,56771,567.71,19136.70,27700,1,29071,0,48.0,2,0,0
2,6368,4,1,0,1,0,29675,296.75,0.00,15994,1,13681,0,77.0,3,0,0
3,3698,3,1,4,4,1526193,292621,2926.21,76309.65,201603,3,91018,1,35.0,1,0,0
4,4933,0,1,3,1,575637,74245,742.45,28781.85,32815,1,41430,0,0.0,0,0,0
5,5218,2,0,0,4,0,194121,1941.21,0.00,161812,4,32309,0,27.0,1,0,0
6,6368,2,3,3,2,639086,125014,1250.14,31954.30,58618,1,66396,1,23.0,1,0,0
7,6368,4,1,0,4,0,301791,3017.91,0.00,166342,1,135449,3,87.0,3,0,0
8,3698,2,1,1,1,272576,33149,331.49,13628.80,5082,1,28067,0,82.0,3,0,0
9,3698,0,1,4,3,919064,80063,800.63,45953.20,23675,2,56388,1,0.0,0,0,0


In [20]:
individual

,first_name,last_name,open_date,individual_id
0,Kayla,Woodard,2003-10-07,8064
1,Grayson,Arroyo,2008-03-02,9376
2,Greyson,Fitzgerald,2000-07-12,4485
3,Kenneth,Delarosa,2016-01-29,2698
4,Jesus,Mckenzie,2005-07-17,9229
5,Benjamin,Sexton,2000-11-26,5106
6,Xander,Mccarty,2020-12-03,8471
7,Austin,Weeks,2016-01-30,6009
8,Leilani,Acosta,2000-03-07,4121
9,Alyssa,Franklin,2009-08-07,6623


In [21]:
participation_type

,member_nbr,participation_type,individual_id
0,8784,101,8471
1,5218,101,8064
2,6368,101,9376
3,3698,101,6623
4,4933,101,4485
5,5218,102,6009
6,6368,102,4121
7,6368,102,9229
8,3698,102,5106
9,3698,102,2698


### Notes

In [ ]:
https://www.newyorkfed.org/microeconomics/databank
    
Debt type	Average balance
Mortgage debt	$35,660
Home equity lines of credit	$1,370
Auto loan	$4,760
Credit card debt	$2,980
Student loan debt	$5,610
Other debt (personal loans, payday loans, etc.)	$1,520

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

mu, sigma = 6.1, 1.3 # mean and standard deviation

s = np.random.lognormal(mu, sigma, 1000)
count, bins, ignored = plt.hist(s, 100, density=True, align='mid')

x = np.linspace(min(bins), max(bins), 10000)

pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))

       / (x * sigma * np.sqrt(2 * np.pi)))

plt.plot(x, pdf, linewidth=2, color='r')

plt.axis('tight')

plt.show()

In [ ]:
mu, sigma = 6.1, 1.35 # mean and standard deviation

s = np.random.lognormal(mu, sigma, 1000)
s.mean(), s.max()

In [ ]:
test = generate_agr_membertotal([12345, 6789])

In [ ]:
test

Determine row wise

#### Savings distributions
Balance
* Minimum 25  
* Maximum 250,000  
* Mean  7,000  

Count
* Minimum 1
* Maximum 5
* Mean 2.5

#### Loan distributions

Balance  
* Minimum 0  
* Maximum 1,000,000  
* Mean 92,000  

Count  
* Minimum 0  
* Maximum 5  
* Mean 


#### Logic for populating fields

Random
CLS_LN_CNT = 0-5  
CLS_SV_CNT = 0-5  

OPN_LN_ALL_CNT = 0-5  
OPN_SV_ALL_CNT = 1-5  

Dependent
OPN_LN_BAL = OPN_LN_ALL_CNT * sample from loan distribution  
OPN_SV_BAL = OPN_SV_ALL_CNT * sample from savings distribution  

DIV_YTD_AMT = open sv bal * 1%  
INT_YTD_AMT = opn ln bal * 5%  

SV_SD_BAL = 25-opn_sv_bal  
SV_SD_CNT = 1-opn_sv_cnt  

SV_CERT_BAL = opn_sv_bal - sv_sd_bal  
SV_CERT_CNT = opn_sv_cnt - sv_sd_cnt  

MAX_DAYS_DELQ_CNT = if cls_ln_cnt > 0: (0 - 90),  2% probability of 90+ else: 0  
MAX_MTHS_DELQ_CNT = max_days_delq_cnt / 30  

WO_LN_CNT = if max days delinquent>90: cls_ln_cnt * random number from loan distribution else 0  
WO_LN_BAL = wo_ln_cnt * (random number from loan distribution)  

ignoring these for now
~SV_REG_BAL = ~
~SV_REG_CNT = 1- opn_sv_all_cnt~

In [ ]:
def generate_agr_membertotal_today(membership_participation):
    Pass
    

    
#columns
CLS_LN_CNT
CLS_SV_CNT
DIV_YTD_AMT
INT_YTD_AMT
OPN_LN_ALL_CNT
OPN_LN_BAL
OPN_SV_ALL_CNT
OPN_SV_BAL
SV_CERT_BAL
SV_CERT_CNT
SV_REG_BAL
SV_REG_CNT
SV_SD_BAL
SV_SD_CNT
WO_LN_BAL
WO_LN_CNT
MAX_DAYS_DELQ_CNT
MAX_MTHS_DELQ_CNT


In [ ]:
def quesadilla_cost_calculator(cheese_cost, slices_in_pack, tortilla_cost, tortillas, slices_per_quesadilla, tortillas_per_quesadilla):
    cost_per_slice = cheese_cost/slices_in_pack
    cost_per_tortilla= tortilla_cost/tortillas
    cost_per_quesadilla = (tortillas_per_quesadilla * cost_per_tortilla) + (slices_per_quesadilla * cost_per_slice)
    return(f'using {slices_per_quesadilla} slice of cheese and {tortillas_per_quesadilla} tortillas each, it is costing you {cost_per_quesadilla} per quesadilla')

In [ ]:
quesadilla_cost_calculator(8, 42, 4, 80, 1, 2)